In [ ]:
# make sure it is done before starting to train
!cd .. && python setup.py install && pip install pytorch_lightning==1.5

In [2]:
import os; os.chdir(r'P:\projects\ncps')

In [3]:
from run import main

%load_ext autoreload
%autoreload 2

In [4]:
main()

P:\projects\ncps\run.py:25: DtypeWarning: Columns (15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


TypeError: 'int' object is not callable

In [ ]:
from run import read_data
from config import Config
import project.utils as utils
import pandas as pd
from torch import Tensor
import pytorch_lightning as pl
import torch.utils.data as data_utils
from sklearn.preprocessing import MinMaxScaler
from project.model import SequenceLearner
from ncps.torch import LTC

In [ ]:
device = "cpu"#"cuda" if torch.cuda.is_available() else "cpu"
# data related section
data_raw = read_data(Config.PATH)
data_raw = utils.prepare_data(data_raw, station=Config.STATION, features=Config.FEATURES_LIST)


test_data = data_raw.copy()
test_data = utils.make_features(test_data, features=Config.FEATURES_LIST)

x_features, y_features = utils.generate_test_data(
    test_data,
    Config.FILTER_DT_TILL,
    features=Config.FEATURES_LIST,
    push_y_by = 60,
    unit = 'd',
)

x_test_scaler = MinMaxScaler()
x_test_scaler, x_features = utils.normalize_data(
    x_test_scaler,
    x_features)

y_test_scaler = MinMaxScaler()
y_test_scaler, y_features = utils.normalize_data(
    y_test_scaler,
    y_features)

out_features = y_features.shape[-1]
in_features = x_features.shape[-1]

ds = data_utils.TensorDataset(
    Tensor(x_features), Tensor(y_features)
)

dataloader = data_utils.DataLoader(
    ds,
    batch_size=Config.BATCH_SIZE,
    num_workers=Config.NUM_WORKERS,
    shuffle=False, 
)

In [ ]:
trainer = SequenceLearner.load_from_checkpoint(
    "/home/vlsta001/git_projects/ncps-slaf/slaf-project/checkpoints/model-lnn_units=16-lr=0.01-num_epochs=10-lnn_modified=True-v1.ckpt")
model = trainer.model
model.to(device)
model.eval()

In [ ]:
prediction = model(Tensor(x_features).view(1, -1, in_features).to(device))[0].detach().numpy()

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme()
plt.figure(figsize=(6, 4))
plt.plot(y_features[:, 0], label="Target output")
plt.plot(prediction[0, :, 0], label="NCP output")

plt.title("Before training")
plt.legend(loc="upper right")
plt.show()

In [ ]:
prediction = utils.denormalize_data(x_test_scaler, x_features)

In [ ]:
prediction

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
ltc_model.rnn_cell._params

In [ ]:
model = SequenceLearner.load_from_checkpoint("/home/vlsta001/git_projects/ncps-slaf/slaf-project/checkpoints/epoch=0-step=311-v14.ckpt")
model.model.to("cuda")
model.model.eval()

In [ ]:
x_features

In [ ]:
# predict with the model
y_hat = model.model(x_features)

In [ ]:
y_hat